In [1]:
import requests
import pandas as pd
import pymysql
from datetime import datetime, timedelta
import traceback2
import time
import urllib.parse

In [2]:
def get_host_url(url):
    parsed_url = urllib.parse.urlparse(url)
    host_breakup = parsed_url.netloc.split('.')
#     print([url, f'{parsed_url.scheme}://{host_breakup[-2]}.{host_breakup[-1]}'])
    url = f'{parsed_url.scheme}://{host_breakup[-2]}.{host_breakup[-1]}'
    return url
    

## CMC Symbols Info

In [3]:
api_key = "a86e3326-8c8d-434e-897a-5d6becbf1f35"

### pull all symbols and id at CMC

In [4]:
def get_cmc_symbols_ids(url):
    payload = {
        'limit':5000,
#         'listing_status': 'active,inactive,untracked',
#         'start':7406
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': api_key
    }
    
    response = requests.get(url, params=payload, headers=headers).json()
    return response

def pull_cmc_cryptocurrency_meta_info(url, cryptocurrency_id):
    payload = {
        'aux':'urls,tags,status',
        'id': cryptocurrency_id
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': api_key
    }
    
    response = requests.get(url, params=payload, headers=headers).json()
    return response

In [5]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
response = get_cmc_symbols_ids(url)

In [6]:
len(response['data'])

5000

In [7]:
def parse_cmc_symbol_ids_response(json_response):
    parsed_cmc_symbol_ids_data = {}
    json_data = response['data']
    for json_obj in json_data:
        parsed_cmc_symbol_ids_data[json_obj['id']] = [json_obj['id'], json_obj['name'], json_obj['symbol'], json_obj['slug'], json_obj['is_active']]
    return parsed_cmc_symbol_ids_data


In [8]:
parsed_cmc_symbol_ids_data = parse_cmc_symbol_ids_response(response)

In [9]:
parsed_cmc_symbol_ids_data

{1: [1, 'Bitcoin', 'BTC', 'bitcoin', 1],
 2: [2, 'Litecoin', 'LTC', 'litecoin', 1],
 3: [3, 'Namecoin', 'NMC', 'namecoin', 1],
 4: [4, 'Terracoin', 'TRC', 'terracoin', 1],
 5: [5, 'Peercoin', 'PPC', 'peercoin', 1],
 8: [8, 'Feathercoin', 'FTC', 'feathercoin', 1],
 10: [10, 'Freicoin', 'FRC', 'freicoin', 1],
 13: [13, 'Ixcoin', 'IXC', 'ixcoin', 1],
 16: [16, 'WorldCoin', 'WDC', 'worldcoin', 1],
 18: [18, 'Digitalcoin', 'DGC', 'digitalcoin', 1],
 25: [25, 'Goldcoin', 'GLC', 'goldcoin', 1],
 35: [35, 'Phoenixcoin', 'PXC', 'phoenixcoin', 1],
 37: [37, 'Megacoin', 'MEC', 'megacoin', 1],
 41: [41, 'Infinitecoin', 'IFC', 'infinitecoin', 1],
 42: [42, 'Primecoin', 'XPM', 'primecoin', 1],
 43: [43, 'Anoncoin', 'ANC', 'anoncoin', 1],
 45: [45, 'CasinoCoin', 'CSC', 'casinocoin', 1],
 50: [50, 'Emerald Crypto', 'EMD', 'emerald', 1],
 52: [52, 'XRP', 'XRP', 'xrp', 1],
 53: [53, 'Quark', 'QRK', 'quark', 1],
 56: [56, 'Zetacoin', 'ZET', 'zetacoin', 1],
 61: [61, 'TagCoin', 'TAG', 'tagcoin', 1],
 64: 

In [10]:
def get_cmc_cryptocurrency_meta_info_by_id(id):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/info'
    response = pull_cmc_cryptocurrency_meta_info(url, id)
    ids_json_data = response['data']
    ids_meta_info = {}
    for cmc_id in ids_json_data:
        json_data = ids_json_data[cmc_id]
        meta_info = []
        meta_info.append(json_data['category'])
        meta_info.append(get_host_url(json_data['urls']['website'][0]) if json_data['urls']['website'] else '')
        meta_info.append(','.join(json_data['tags']) if json_data['tags'] else '')
        ids_meta_info[f'{cmc_id}'] = meta_info
    print(len(ids_meta_info))
    return ids_meta_info
    

In [11]:
cmc_cryptocurrency_data = []
cryptocurrency_count = len(parsed_cmc_symbol_ids_data)
start_index = 0
batch_size = 100
cryptocurrency_ids = list(parsed_cmc_symbol_ids_data.keys())
cryptocurrency_ids = [str(cryptocurrency_id) for cryptocurrency_id in cryptocurrency_ids]
while start_index < cryptocurrency_count:
    end_index = start_index + batch_size
    cryptocurrency_ids_subset = cryptocurrency_ids[start_index : end_index]
    cryptocurrency_ids_meta_info = get_cmc_cryptocurrency_meta_info_by_id(','.join(cryptocurrency_ids_subset))
    for cryptocurrency_id in cryptocurrency_ids_subset:
        cryptocurrency_data = []
        cryptocurrency_data.extend(parsed_cmc_symbol_ids_data[int(cryptocurrency_id)])
        cryptocurrency_data.extend(cryptocurrency_ids_meta_info[(cryptocurrency_id)])
        cmc_cryptocurrency_data.append(cryptocurrency_data)
    time.sleep(1)
    start_index = end_index

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [12]:
cmc_cryptocurrency_data

[[1,
  'Bitcoin',
  'BTC',
  'bitcoin',
  1,
  'coin',
  'https://bitcoin.org',
  'mineable,pow,sha-256,store-of-value,state-channels,coinbase-ventures-portfolio,three-arrows-capital-portfolio,polychain-capital-portfolio,binance-labs-portfolio,arrington-xrp-capital,blockchain-capital-portfolio,boostvc-portfolio,cms-holdings-portfolio,dcg-portfolio,dragonfly-capital-portfolio,electric-capital-portfolio,fabric-ventures-portfolio,framework-ventures,galaxy-digital-portfolio,huobi-capital,alameda-research-portfolio,a16z-portfolio,1confirmation-portfolio,winklevoss-capital,usv-portfolio,placeholder-ventures-portfolio,pantera-capital-portfolio,multicoin-capital-portfolio,paradigm-xzy-screener'],
 [2,
  'Litecoin',
  'LTC',
  'litecoin',
  1,
  'coin',
  'https://litecoin.org',
  'mineable,pow,scrypt,medium-of-exchange,binance-chain'],
 [3,
  'Namecoin',
  'NMC',
  'namecoin',
  1,
  'coin',
  'https://namecoin.org',
  'mineable,pow,sha-256,platform'],
 [4,
  'Terracoin',
  'TRC',
  'terracoin

In [13]:
coins_info_df = pd.DataFrame(cmc_cryptocurrency_data, columns=['id', 'name', 'symbol', 'slug', 'is_active', 'category', 'url', 'tags'])
now = datetime.now()
now = datetime(now.year, now.month, now.day)
coins_info_df['date'] = now

In [14]:
coins_info_df['current_name'] = coins_info_df['name']

In [23]:
coins_info_df

,id,name,symbol,slug,is_active,category,url,tags,date,current_name
0,1,Bitcoin,BTC,bitcoin,1,coin,https://bitcoin.org,"mineable,pow,sha-256,store-of-value,state-chan...",2021-05-14,Bitcoin
1,2,Litecoin,LTC,litecoin,1,coin,https://litecoin.org,"mineable,pow,scrypt,medium-of-exchange,binance...",2021-05-14,Litecoin
2,3,Namecoin,NMC,namecoin,1,coin,https://namecoin.org,"mineable,pow,sha-256,platform",2021-05-14,Namecoin
3,4,Terracoin,TRC,terracoin,1,coin,http://terracoin.io,"mineable,pow,sha-256,masternodes",2021-05-14,Terracoin
4,5,Peercoin,PPC,peercoin,1,coin,http://peercoin.net,"mineable,hybrid-pow-pos,sha-256,medium-of-exch...",2021-05-14,Peercoin
...,...,...,...,...,...,...,...,...,...,...
4995,9582,DisCas Vision,DISC,discas-vision,1,token,https://discas.vision,,2021-05-14,DisCas Vision
4996,9583,Melalie,MEL,melalie,1,token,https://melalie.com,,2021-05-14,Melalie
4997,9584,Slam Token,SLAM,slam-token,1,token,https://slamtoken.com,,2021-05-14,Slam Token
4998,9585,XBN Community Token,XBC,xbn-community-token,1,token,https://xbn.finance,,2021-05-14,XBN Community Token


In [26]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

def insert_symbols_meta_info(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            dt = row["date"].timestamp()
            sql = "INSERT INTO `cmc_coin_metainfo` (`cmc_id`, `name`, `current_name`, `symbol`, `slug`, `is_active`, `category`, `url`, `tags`, `timestamp_dt`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, FROM_UNIXTIME(%s))"
            cursor.execute(sql, (row["id"], row["name"], row["current_name"], row["symbol"], row["slug"], row["is_active"], row["category"], row["url"], row["tags"], dt))
            print(row["date"], row["id"], row["name"], row["current_name"], row["symbol"], row["slug"], row["is_active"], row["category"], row["url"], row["tags"])
        except Exception:
            print('got Exception')
#             print(traceback2.format_exc())
#             pass
    connection.commit()
    cursor.close()
    connection.close()

In [27]:
insert_symbols_meta_info(coins_info_df)

got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 16 WorldCoin WorldCoin WDC worldcoin 1 coin http://worldcoin.global mineable,pow,scrypt
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Except

got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Ex

got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 3712 Cloudbric Cloudbric CLBK cloudbric 1 token https://cloudbric.io 
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 3761 DISCIPLINA DISCIPLINA DSCPL disciplina 1 token https://disciplina.io 
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got

got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 4848 The Forbidden Forest The Forbidden Forest FORESTPLUS the-forbidden-forest 1 coin https://theforbiddenforest.org 
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 4865 Nahmii Nahmii NII nahmii 1 token https://nahmii.io 
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
g

got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 5899 Casper Casper CSPR casper 1 coin https://casper.network enterprise-solutions,hashkey-capital-portfolio
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 5925 Pkoin Pkoin PKOIN pocketnet 1 coin https://pocketnet.app 
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got E

...

2021-05-14 00:00:00 9046 8PAY 8PAY 8PAY 8pay 1 token https://8pay.network 
got Exception
2021-05-14 00:00:00 9048 Xpool Xpool XPO xpool 1 token https://xpool.app binance-smart-chain
2021-05-14 00:00:00 9049 Gorilla-Fi Gorilla-Fi GFI gorilla-fi 1 token https://gorillafi.com 
2021-05-14 00:00:00 9050 Delta Exchange Token Delta Exchange Token DETO delta-exchange-token 1 token https://delta.exchange 
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 9059 Mero Mero MERO mero 1 token https://mero.blue 
got Exception
got Exception
got Exception
2021-05-14 00:00:00 9063 Tcake Tcake TCAKE pancaketools 1 token https://tcake.io 
2021-05-14 00:00:00 9064 Sting Defi Sting Defi SDFI sting-defi 1 token https://stingdefi.com 
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
got Exception
2021-05-14 00:00:00 9075 Bafi Finance Bafi Finance BAFI bafi-finance 1 token https://bafi.finance 
got Exception
g

2021-05-14 00:00:00 9185 FairEclipse FairEclipse FECLIPSE faireclipse 1 token https://faireclipse.com 
2021-05-14 00:00:00 9186 Dark Matter Dark Matter DMT dark-matter 1 token https://darkmatter.finance 
2021-05-14 00:00:00 9187 MY IDENTITY COIN MY IDENTITY COIN MYID my-identity-coin 1 token https://okglobalcoinsg.com 
2021-05-14 00:00:00 9188 Globe Derivative Exchange Globe Derivative Exchange GDT globe-derivative-exchange 1 token https://globedx.com centralized-exchange,derivatives
2021-05-14 00:00:00 9191 Occam.Fi Occam.Fi OCC occamfi 1 token https://occam.fi 
2021-05-14 00:00:00 9192 Kambria Yield Tuning Engine Kambria Yield Tuning Engine KYTE kambria-yield-tuning-engine 1 token https://kambria.io 
2021-05-14 00:00:00 9193 Prostarter Prostarter PROT prostarter 1 token https://prostarter.io 
2021-05-14 00:00:00 9194 Saito Saito SAITO saito 1 token https://saito.io 
2021-05-14 00:00:00 9196 Genesis Shards Genesis Shards GS genesis-shards 1 token https://genshards.com collectibles-nft

2021-05-14 00:00:00 9302 MoMo KEY MoMo KEY KEY momo-key 1 token https://mobox.io collectibles-nfts,gaming
2021-05-14 00:00:00 9303 Save Planet Earth Save Planet Earth SPE save-planet-earth 1 token https://saveplanetearth.io 
2021-05-14 00:00:00 9308 Vulcan Forged PYR Vulcan Forged PYR PYR vulcan-forged-pyr 1 token https://vulcanforged.com collectibles-nfts,gaming,entertainment
2021-05-14 00:00:00 9309 HyperChain HyperChain HYPER hyperchain 1 token https://hyperchainx.com 
2021-05-14 00:00:00 9312 The Lab Finance The Lab Finance LABO the-lab-finance 1 token https://thelab.finance 
2021-05-14 00:00:00 9316 Shipit pro Shipit pro SHPP shipit-pro 1 token https://shipitpro.io 
2021-05-14 00:00:00 9317 Daxhund Daxhund DXH daxhund 1 token https://daxhund.app 
2021-05-14 00:00:00 9318 BeforeCoinMarketCap BeforeCoinMarketCap BCMC1 beforecoinmarketcap 1 token https://beforecoinmarketcap.com 
2021-05-14 00:00:00 9320 KILIMANJARO KILIMANJARO KILI kilimanjaro 1 token https://kilimanjaro.finance 
202

2021-05-14 00:00:00 9443 Step Finance Step Finance STEP step-finance 1 token https://step.finance solana-ecosystem
2021-05-14 00:00:00 9446 Lottery Token Lottery Token LOT lottery-token 1 token https://lotterytoken.net 
2021-05-14 00:00:00 9449 Sienna (ERC20) Sienna (ERC20) wSIENNA sienna-erc20 1 token https://sienna.network 
2021-05-14 00:00:00 9450 BLACKHOLE PROTOCOL BLACKHOLE PROTOCOL BLACK blackhole-protocol 1 token https://blackhole.black 
2021-05-14 00:00:00 9451 Verso Token Verso Token VSO verso-token 1 token https://verso.finance 
2021-05-14 00:00:00 9452 Bandot Protocol Bandot Protocol BDT bandot-protocol 1 token https://bandot.io 
2021-05-14 00:00:00 9453 Agave Agave AGVE agave 1 token https://agave.finance lending-borowing
2021-05-14 00:00:00 9455 Lemond Lemond LEMD lemond 1 token https://lemond.money 
2021-05-14 00:00:00 9456 Australian Safe Shepherd Australian Safe Shepherd ASS australian-safe-shepherd 1 token https://assfinance.com memes,binance-smart-chain
2021-05-14 00:

2021-05-14 00:00:00 9582 DisCas Vision DisCas Vision DISC discas-vision 1 token https://discas.vision 
2021-05-14 00:00:00 9583 Melalie Melalie MEL melalie 1 token https://melalie.com 
2021-05-14 00:00:00 9584 Slam Token Slam Token SLAM slam-token 1 token https://slamtoken.com 
2021-05-14 00:00:00 9585 XBN Community Token XBN Community Token XBC xbn-community-token 1 token https://xbn.finance 
2021-05-14 00:00:00 9586 PRIVATEUM INITIATIVE PRIVATEUM INITIATIVE PVM privateum-initiative 1 token https://privateum.org 


In [20]:
coins_info_df[coins_info_df['symbol']== 'JUL']

,id,name,symbol,slug,is_active,category,url,tags,date,current_name
2982,6937,JustLiquidity,JUL,justliquidity,1,token,https://justliquidity.org,"ethereum,binance-smart-chain",2021-05-14,JustLiquidity


In [19]:
coins_info_df.to_csv('cmc_cryptocurrency_meta_info.csv')

In [20]:
df = coins_info_df.groupby([ 'symbol']).count()

In [21]:
df = df[df['id'] > 1]

In [22]:
symbols = list(df.index.values)
symbols

['AAPL',
 'ACE',
 'ACM',
 'ACOIN',
 'ACU',
 'AIN',
 'ALG',
 'AMZN',
 'ANC',
 'ANK',
 'APE',
 'APL',
 'AST',
 'AT',
 'ATP',
 'AUR',
 'AUTO',
 'BABA',
 'BANANA',
 'BANK',
 'BBC',
 'BBP',
 'BCA',
 'BCP',
 'BCZ',
 'BDP',
 'BDT',
 'BEAR',
 'BEE',
 'BERRY',
 'BET',
 'BFC',
 'BFI',
 'BID',
 'BIFI',
 'BILI',
 'BIRD',
 'BIT',
 'BITS',
 'BLINK',
 'BLOC',
 'BNTX',
 'BOND',
 'BOS',
 'BOX',
 'BPT',
 'BSC',
 'BSD',
 'BST',
 'BTA',
 'BTCT',
 'BTCV',
 'BTR',
 'BTS',
 'BTU',
 'BULL',
 'BUNNY',
 'BUX',
 'BXC',
 'BYND',
 'CAN',
 'CAP',
 'CBC',
 'CEN',
 'CGT',
 'CHESS',
 'CHI',
 'CMCT',
 'CMT',
 'CNT',
 'COFI',
 'COMB',
 'COMP',
 'CON',
 'CORN',
 'CPAY',
 'CPC',
 'CPU',
 'CRE',
 'CREDIT',
 'CRP',
 'CTC',
 'CTT',
 'CUB',
 'CURE',
 'CVR',
 'CVT',
 'DAPP',
 'DEC',
 'DFS',
 'DFT',
 'DFX',
 'DG',
 'DGN',
 'DICE',
 'DIP',
 'DMD',
 'DNA',
 'DON',
 'DOS',
 'DRAGON',
 'DRC',
 'DRM',
 'DSD',
 'DT',
 'DUCK',
 'DUO',
 'DVS',
 'EC',
 'ECU',
 'EDC',
 'EDG',
 'EGG',
 'EM',
 'EMC',
 'EPS',
 'ETHV',
 'EVC',
 'EXP',
 'FAIR

In [23]:
coins_info_df[(coins_info_df['symbol'].isin(symbols))]

,id,name,symbol,slug,is_active,category,url,tags,current_name
4,5,Peercoin,PPC,peercoin,1,coin,http://peercoin.net,"mineable,hybrid-pow-pos,sha-256,medium-of-exch...",Peercoin
8,16,WorldCoin,WDC,worldcoin,1,coin,http://worldcoin.global,"mineable,pow,scrypt",WorldCoin
11,35,Phoenixcoin,PXC,phoenixcoin,1,coin,http://phoenixcoin.org,"mineable,neoscrypt",Phoenixcoin
15,43,Anoncoin,ANC,anoncoin,1,coin,https://anoncoin.net,"mineable,pow,privacy",Anoncoin
22,64,FLO,FLO,flo,1,coin,http://flo.cash,"mineable,pow,scrypt",FLO
...,...,...,...,...,...,...,...,...,...
4581,9086,Minter HUB,HUB,minter-hub,1,token,http://minter.network,,Minter HUB
4586,9091,CPCoin,CPC,cpcoin,1,coin,https://cpcoin.io,,CPCoin
4590,9097,Try.Finance,TRY,try-finance,1,token,https://try.finance,,Try.Finance
4613,9137,UNITi Protocol,UNIT,uniti-protocol,1,token,https://unitiprotocol.com,,UNITi Protocol


In [24]:
coins_info_df[(coins_info_df['symbol'] == 'PPC')]

,id,name,symbol,slug,is_active,category,url,tags,current_name
4,5,Peercoin,PPC,peercoin,1,coin,http://peercoin.net,"mineable,hybrid-pow-pos,sha-256,medium-of-exch...",Peercoin
2652,6253,PHILLIPS PAY COIN,PPC,philips-pay-coin,1,token,http://p-pay.io,,PHILLIPS PAY COIN


SHeet Import: Symbol Mappings.

In [82]:
def insert_tt_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_tt']])
            sql = "INSERT INTO `tt_symbols` (`symbol`, `name_in_cmc`, `name_in_tt`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_tt']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_debank_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_debank']])
            sql = "INSERT INTO `debank_symbols` (`symbol`, `name_in_cmc`, `name_in_debank`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_debank']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_defi_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_defi']])
            sql = "INSERT INTO `defi_symbols` (`symbol`, `name_in_cmc`, `name_in_defi`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_defi']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_duneanalytics_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_duneanalytics'], row['query']])
            sql = "INSERT INTO `dune_symbols` (`symbol`, `name_in_cmc`, `name_in_duneanalytics`, `query`) VALUES (%s, %s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_duneanalytics'],  ""))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_defistation_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_defistation']])
            sql = "INSERT INTO `defistation_symbols` (`symbol`, `name_in_cmc`, `name_in_defistation`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_defistation']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

def insert_solana_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_solana']])
            sql = "INSERT INTO `solana_symbols` (`symbol`, `name_in_cmc`, `name_in_solana`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_solana']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()
    return

In [76]:
df = pd.read_csv('solana_defi_mappings.csv')

In [77]:
df

,name,current_name,symbol,name_in_duneanalytics,name_in_debank,name_in_defi,name_in_tt,name_in_defistation,name_in_solana
0,NaN,NaN,NaN,dYdX,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,InstaDApp,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Set Protocol,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1sd,NaN,NaN,NaN,NaN
4,Aave,Aave,AAVE,NaN,Aave AMM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
176,ORCA Alliance,ORCA Alliance,ORCA,NaN,NaN,NaN,NaN,NaN,Orca
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serum Swap
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SerumDex
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unlimitedcope


In [57]:
df[df['name_in_tt'].notna() & df['symbol'].notna()][['symbol', 'current_name']]

,symbol,current_name
249,FLX,Reflexer Ungovernance Token


In [68]:
def get_source_mapping_df(df, source):
    source = f"name_in_{source}"
    df = df[df[source].notna() & df['symbol'].notna()][['symbol', 'current_name', source]].copy()
    df.columns = ['symbol', 'name_in_cmc', source]
    print(df.columns)
    return df

In [59]:
get_source_mapping_df(df, 'duneanalytics')

,symbol,name_in_cmc,name_in_duneanalytics
1,DEFI,Defi,DeFi


In [60]:
sources = ['duneanalytics', 'tt', 'debank', 'defi']

In [84]:
insert_tt_coin_cmc_symbol(get_source_mapping_df(df, 'tt'))
# insert_duneanalytics_coin_cmc_symbol(get_source_mapping_df(df, 'duneanalytics'))
# insert_debank_coin_cmc_symbol(get_source_mapping_df(df, 'debank'))
insert_defi_coin_cmc_symbol(get_source_mapping_df(df, 'defi'))
# insert_defistation_coin_cmc_symbol(get_source_mapping_df(df, 'defistation'))
# insert_solana_coin_cmc_symbol(get_source_mapping_df(df, 'solana'))

Index(['symbol', 'name_in_cmc', 'name_in_tt'], dtype='object')
['ALCX', 'Alchemix', 'Alchemix Finance']
['ALPHA', 'Alpha Finance Lab', 'Alpha Finance']
['BADGER', 'Badger DAO', 'BadgerDAO']
['BASK', 'BasketDAO', 'Basket DAO']
['CFG', 'Centrifuge', 'Centrifuge']
['CRV', 'Curve DAO Token', 'Curve']
Traceback (most recent call last):
  File "<ipython-input-82-282a9632b5f1>", line 8, in insert_tt_coin_cmc_symbol
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/home/ubuntu/.local/lib/python3

tt_symbols updates

In [72]:
tts2 = pd.read_sql_query("select * from tt_symbols2", __connect__())

In [73]:
tts2

,symbol,name_in_cmc,name_in_tt
0,,,Alchemix Finance
1,,,Alpha Finance
2,,,Autofarm
3,,,BadgerDAO
4,,,Basket DAO
...,...,...,...
89,XTZ,Tezos,Tezos
90,XVS,Venus,Venus
91,YFI,yearn.finance,yearn.finance
92,ZEC,Zcash,Zcash


In [75]:
insert_tt_coin_cmc_symbol(tts2)

['', '', 'Alchemix Finance']
['', '', 'Alpha Finance']
['', '', 'Autofarm']
['', '', 'BadgerDAO']
['', '', 'Basket DAO']
['', '', 'Binance Chain']
['', '', 'Centrifuge']
['', '', 'Curve']
['', '', 'dHEDGE']
['', '', 'dYdX']
Traceback (most recent call last):
  File "<ipython-input-66-0c9ea39b28fc>", line 8, in insert_tt_coin_cmc_symbol
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pymysql/connections.py", line 1156, in read
    first_pac

## Debank Cryptocurrency Meta Info

In [25]:
import requests
import pandas as pd

In [26]:
def get_data_from_host(url):
    r = requests.get(url)
    return r.json()

In [27]:
project_list_url = 'https://api.debank.com/project/v2/list'
project_list_response = get_data_from_host(project_list_url)

In [28]:
project_list_response['data'][0]

{'active_user_count_24h': 1067,
 'chain': 'eth',
 'chain_info_dict': {'id': 'eth',
  'logo_url': 'https://static.debank.com/image/chain/logo_url/eth/6e0cd1f895af9836ee8c32cfc03bc279.png',
  'name': {'ch': '以太坊', 'en': 'Ethereum'},
  'token_id': 'eth'},
 'contract_call_count_24h': 9993,
 'create_at': 1588847817540.0,
 'id': '0x',
 'is_stable': False,
 'is_tvl': False,
 'is_visible': True,
 'lock_underlying_usd': None,
 'lock_usd': None,
 'logo_url': 'https://static.debank.com/image/project/logo_url/0x/a97d06e29468082e0859f3cd48d5a2ec.png',
 'name': {'ch': '0x', 'en': '0x'},
 'platform_token_chain': 'eth',
 'platform_token_id': '0xe41d2489571d322189246dafa5ebde1f4699f498',
 'priority': 9,
 'publish_at': 1502809252.0,
 'site_url': 'https://0x.org',
 'tag_ids': ['dex'],
 'total_contract_count': 7,
 'total_user_count': 52200,
 'tvl': 0,
 'update_at': 1613718897246.0}

In [29]:
debank_projects_info = []
for project_response in project_list_response['data']:
    debank_projects_info.append([project_response['id'], project_response['name']['en'], get_host_url(project_response['site_url'])])

In [30]:
debank_projects_info_df = pd.DataFrame(debank_projects_info, columns=['id', 'name', 'url'])

In [31]:
debank_projects_info_df

,id,name,url
0,0x,0x,https://0x.org
1,1inch,1inch V1,https://1inch.exchange
2,1inch2,1inch,https://1inch.exchange
3,1sd,1sd,https://1sd.finance
4,88mph,88mph V1,https://88mph.app
...,...,...,...
307,zai,ZAI V1,https://zai.finance
308,zai2,ZAI V2,https://zai.fi
309,zelda,Zelda,https://zelda.cash
310,zeroset,Zero Set Dollar,https://zeroset.finance


In [32]:
debank_unique_urls = list(debank_projects_info_df.url.unique())

In [33]:
len(debank_unique_urls)

282

### debank's cryptocurrencies available at CMC : Count

In [110]:
def insert_debank_coin_cmc_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_debank']])
            sql = "INSERT INTO `debank_symbols2` (`symbol`, `name_in_cmc`, `name_in_debank`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_debank']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [34]:
len(coins_info_df[(coins_info_df['url'].isin(debank_unique_urls))]['url'].unique())

188

In [114]:
coins_info_df[(coins_info_df['url'].isin(debank_unique_urls))]

,id,name,symbol,slug,is_active,category,url,tags,current_name
290,1552,Enzyme,MLN,enzyme,1,token,https://enzyme.finance,"asset-management,defi,kinetic-capital,placehol...",Enzyme
321,1659,Gnosis,GNO,gnosis-gno,1,token,https://gnosis.io,"services,decentralized-exchange,defi,predictio...",Gnosis
329,1680,Aragon,ANT,aragon,1,token,https://aragon.org,"platform,cosmos-ecosystem,enterprise-solutions...",Aragon
354,1727,Bancor,BNT,bancor,1,token,https://bancor.network,"marketplace,decentralized-exchange,defi,paymen...",Bancor
376,1776,MCO,MCO,crypto-com,1,token,https://crypto.com,medium-of-exchange,MCO
...,...,...,...,...,...,...,...,...,...
4537,9025,Tribe,TRIBE,tribe,1,token,https://fei.money,"defi,governance",Tribe
4545,9034,pETH18C,pETH18C,peth18c,1,token,https://poolin.fi,,pETH18C
4553,9043,Stone DeFi,STN,stone-token,1,token,https://stonedefi.io,,Stone DeFi
4568,9067,Olympus,OHM,olympus,1,token,https://eth.link,,Olympus


In [66]:
sql = "select * from master_symbols where name_in_debank != ''"
maaster_symbol_df = pd.read_sql_query(sql, __connect__())

In [67]:
maaster_symbol_df

,symbol,name_in_cmc,name_in_debank,name_in_tt,name_in_defi,name_in_duneanalytics
0,1INCH,1inch,1inch,1inch,1inch-Liquidity-Protocol,1inch
1,1INCH,1inch,1inch2,1inch,1inch-Liquidity-Protocol,1inch
2,AAVE,Aave,aave,Aave,Aave,Aave
3,AAVE,Aave,Aave V1,Aave,Aave,Aave
4,AAVE,Aave,Aave V2,Aave,Aave,Aave
...,...,...,...,...,...,...
90,YFI,Yearn,Yearn,yearn.finance,yearn.finance,yearn.finance
91,YFI,Yearn,Yearn V2,yearn.finance,yearn.finance,yearn.finance
92,YFII,DFI.money,DFI.Money (YFII),,DFI.money,
93,YLD,Yield,YIELD (yield.is),,,


In [68]:
def get_symbol_and_name_for_url(cmc_df, url, name, master_symbol_df):
    cmc_df = cmc_df[cmc_df['url'] == url]
    if cmc_df.shape[0] == 1:
        return [cmc_df.iloc[0]['name'], cmc_df.iloc[0]['symbol']]
    elif name in master_symbol_df['name_in_debank'].values:
        master_symbol_df = master_symbol_df[master_symbol_df['name_in_debank'] == name]
        if master_symbol_df.shape[0] == 1:
            return [master_symbol_df.iloc[0]['name_in_cmc'], master_symbol_df.iloc[0]['symbol']]
        else:
            return ['']*2
    else:
        return ['', '']

In [69]:
deban_cmc_symbol_map = []
for index, row in debank_projects_info_df.iterrows():
    name = row['name']
    url = row['url']
    debank_mapping = [name]
    debank_mapping.extend(get_symbol_and_name_for_url(coins_info_df, url, name, maaster_symbol_df))
    deban_cmc_symbol_map.append(debank_mapping)

In [115]:
deban_cmc_symbol_map

[['0x', '0x', 'ZRX'],
 ['1inch V1', '', ''],
 ['1inch', '1inch', '1INCH'],
 ['1sd', '', ''],
 ['88mph V1', '88mph', 'MPH'],
 ['88mph V2', '88mph', 'MPH'],
 ['Aave V1', 'Aave', 'AAVE'],
 ['Aave V2', 'Aave', 'AAVE'],
 ['Aavegotchi', '', ''],
 ['Aave AMM', '', ''],
 ['acBTC', '', ''],
 ['ACO', '', ''],
 ['Airswap', 'AirSwap', 'AST'],
 ['Alchemix', 'Alchemix', 'ALCX'],
 ['Alpaca City', 'Alpaca City', 'ALPA'],
 ['Alpha Homora', 'Alpha Homora', 'ALPHA'],
 ['Alpha Homora V2', '', ''],
 ['Ampleforth', 'Ampleforth', 'AMPL'],
 ['AnySwap', 'Anyswap', 'ANY'],
 ['APWine', '', ''],
 ['APY.Finance', 'APY.Finance', 'APY'],
 ['Aragon', '', ''],
 ['ARCx', '', ''],
 ['Armor', 'Armor', 'ARMOR'],
 ['Artem', '', ''],
 ['Augur', '', ''],
 ['Badger DAO', 'Badger', 'BADGER'],
 ['Balancer', '', ''],
 ['Bancor', '', ''],
 ['Band', 'Band Protocol', 'BAND'],
 ['BarnBridge', 'BarnBridge', 'BOND'],
 ['Base', 'Base Protocol', 'BASE'],
 ['Basis Cash', 'Basis Cash', 'BAC'],
 ['Basis Coin', '', ''],
 ['Basis Dollar', ''

In [116]:
df = pd.DataFrame(deban_cmc_symbol_map, columns=['name_in_debank', 'name_in_cmc', 'symbol'])

In [117]:
df = df.drop_duplicates()

In [118]:
df

,name_in_debank,name_in_cmc,symbol
0,0x,0x,ZRX
1,1inch V1,,
2,1inch,1inch,1INCH
3,1sd,,
4,88mph V1,88mph,MPH
...,...,...,...
307,ZAI V1,Zero Collateral Dai,ZAI
308,ZAI V2,,
309,Zelda,,
310,Zero Set Dollar,,


In [75]:
sql = "select * from debank_symbols where name_in_cmc != ''"
debank_symbol_df = pd.read_sql_query(sql, __connect__())

In [119]:
debank_symbol_df

,symbol,name_in_cmc,name_in_debank
0,1INCH,1inch,1inch
1,1INCH,1inch,1INCH Token
2,1INCH,1inch,1inch V1
3,AAVE,Aave,Aave Token
4,AAVE,Aave,Aave V1
...,...,...,...
244,ZAI,Zero Collateral Dai,Zero Collateral Dai
245,ZKS,ZKSwap,Zks
246,ZKS,ZKSwap,ZKSwap
247,ZRX,0x,0x


In [120]:
df3 = debank_symbol_df.append(df)

In [122]:
df3 = df3.drop_duplicates()

,symbol,name_in_cmc,name_in_debank
0,1INCH,1inch,1inch
1,1INCH,1inch,1INCH Token
2,1INCH,1inch,1inch V1
3,AAVE,Aave,Aave Token
4,AAVE,Aave,Aave V1
...,...,...,...
307,ZAI,Zero Collateral Dai,ZAI V1
308,,,ZAI V2
309,,,Zelda
310,,,Zero Set Dollar


In [125]:
insert_debank_coin_cmc_symbol(df3)

['1INCH', '1inch', '1inch']
['1INCH', '1inch', '1INCH Token']
['1INCH', '1inch', '1inch V1']
['AAVE', 'Aave', 'Aave Token']
['AAVE', 'Aave', 'Aave V1']
['AAVE', 'Aave', 'Aave V2']
['ALCX', 'Alchemix', 'Alchemix']
['ALPA', 'Alpaca City', 'Alpaca City']
['ALPA', 'Alpaca City', 'AlpaToken']
['ALPACA', 'Alpaca Finance', 'Alpaca Finance']
['ALPHA', 'Alpha Finance Lab', 'AlphaToken']
['AMP', 'Amp', 'Amp']
['AMPL', 'Ampleforth', 'Ampleforth']
['ANT', 'Aragon', 'Aragon']
['ANT', 'Aragon', 'Aragon Network Token']
['ANY', 'Anyswap', 'Anyswap']
['APY', 'APY.Finance', 'APY Governance Token']
['APY', 'APY.Finance', 'APY.Finance']
['ARMOR', 'ARMOR', 'Armor']
['AST', 'AirSwap', 'AirSwap Token']
['AST', 'AntiscamToken', 'AirSwap Token']
['BAC', 'Basis Cash', 'Basis Cash']
['BADGER', 'Badger DAO', 'Badger DAO']
['BAG', 'Basis Gold', 'Basis Gold']
['BAL', 'Balancer', 'Balancer']
['BAND', 'Band Protocol', 'BandToken']
['BAS', 'Basis Share', 'BAS']
['BASE', 'Base Protocol', 'Base Protocol']
['BCS', 'Basis

['VALUE', 'Value Liquidity', 'Value Liquidity']
['VSP', 'Vesper', 'Vesper']
['xBTC', 'xBTC', 'xBTC']
['XTK', 'xToken', 'xToken']
['XVIX', 'XVIX', 'XVIX']
['XVS', 'Venus', 'Venus']
['YAM', 'YAM v1', 'YAM']
['YAM', 'YAM v3', 'YAM']
['YAX', 'yAxis', 'yAxis']
['Yf-DAI', 'YFDAI.FINANCE', 'YfDAI.finance']
['YFI', 'yearn.finance', 'yearn.finance']
['YFII', 'DFI.Money', 'YFII.finance']
['YFIII', 'DiFy.Finance', 'DiFy.Finance']
['YFIII', 'YFIII', 'DiFy.Finance']
['YFL', 'YF Link', 'YFLink']
['YVS', 'YVS.Finance', 'YVS.Finance']
['ZAI', 'Zero Collateral Dai', 'Zero Collateral Dai']
['ZKS', 'ZKSwap', 'Zks']
['ZKS', 'ZKSwap', 'ZKSwap']
['ZRX', '0x', '0x']
['ZRX', '0x', '0x Protocol Token']
['ZRX', '0x', '0x']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarus

['', '', 'Aave AMM']
['', '', 'acBTC']
['', '', 'ACO']
['AST', 'AirSwap', 'Airswap']
['ALCX', 'Alchemix', 'Alchemix']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in

['', '', 'Balancer']
['', '', 'Bancor']
['BAND', 'Band Protocol', 'Band']
['BOND', 'BarnBridge', 'BarnBridge']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
 

['BELT', 'Belt', 'Belt.fi']
['', '', 'BnEX']
['BSWAP', 'BSCswap', 'BSCswap']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection.

...

['', '', 'Saturn']
['SFG', 'S.Finance', 'S.Finance']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/

Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python

['', '', 'TokenSets']
['TORN', 'Tornado Cash', 'Tornado']
['', '', 'Totle']
['', '', 'TRAMS']
['TRU', 'TrueFi', 'TrueFi']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156

['USF', 'Unslashed Finance', 'Unslashed']
['VSP', 'Vesper', 'Vesper']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_p

['YF-DAI', 'YFDAI.FINANCE', 'YfDAI']
['YFII', 'DFI.Money', 'DFI.Money (YFII)']
['', '', 'YFLink']
['', '', 'Value DeFi']
['', '', 'YIELD (yield.is)']
['YVS', 'YVS.Finance', 'YVS']
['ZAI', 'Zero Collateral Dai', 'ZAI V1']
['', '', 'ZAI V2']
['', '', 'Zelda']
['', '', 'Zero Set Dollar']
['', '', 'ZKSwap']


In [92]:
not_in_df = debank_symbol_df[~debank_symbol_df['name_in_debank'].isin(df['name_in_debank'].values)]

In [82]:
df[df['symbol'] == '1INCH']

,name_in_debank,name_in_cmc,symbol
2,1inch,1inch,1INCH
43,1inch,1inch,1INCH


In [93]:
not_in_df

,symbol,name_in_cmc,name_in_debank
1,1INCH,1inch,1INCH Token
3,AAVE,Aave,Aave Token
8,ALPA,Alpaca City,AlpaToken
10,ALPHA,Alpha Finance Lab,AlphaToken
11,AMP,Amp,Amp
...,...,...,...
241,YFIII,YFIII,DiFy.Finance
243,YVS,YVS.Finance,YVS.Finance
244,ZAI,Zero Collateral Dai,Zero Collateral Dai
245,ZKS,ZKSwap,Zks


In [98]:
debank_symbol_df.sort_values(['name_in_debank'])

,symbol,name_in_cmc,name_in_debank
247,ZRX,0x,0x
248,ZRX,0x,0x Protocol Token
1,1INCH,1inch,1INCH Token
0,1INCH,1inch,1inch
2,1INCH,1inch,1inch V1
...,...,...,...
200,STAKE,xDai,xDai
196,SIG,xSigma,xSigma
231,XTK,xToken,xToken
236,YAX,yAxis,yAxis


In [101]:
debank_valid_values_df = df[df['name_in_cmc'] != ''].sort_values(['name_in_debank'])
debank_valid_values_df

,name_in_debank,name_in_cmc,symbol
0,0x,0x,ZRX
2,1inch,1inch,1INCH
4,88mph V1,88mph,MPH
5,88mph V2,88mph,MPH
214,ADD.XYZ,Add.xyz,ADD
...,...,...,...
284,xBTC,xBTC,xBTC
285,xDai,Wrapped XDAI,wxDai
293,xToken,xToken,XTK
295,xvix,XVIX,XVIX


In [99]:
not_in_df.sort_values(['name_in_debank'])

,symbol,name_in_cmc,name_in_debank
248,ZRX,0x,0x Protocol Token
1,1INCH,1inch,1INCH Token
148,MPH,88mph,88mph.app
149,MPH,Morpher,88mph.app
16,APY,APY.Finance,APY Governance Token
...,...,...,...
174,PNT,pNetwork,pNetwork Token
172,PNT,Penta,pNetwork Token
181,RFI,reflect.finance,reflect.finance
196,SIG,xSigma,xSigma


In [104]:
df2 = debank_valid_values_df.append(not_in_df)

In [107]:
df2 = df2.drop_duplicates()
df2.sort_values(['name_in_debank'])

,name_in_debank,name_in_cmc,symbol
0,0x,0x,ZRX
248,0x Protocol Token,0x,ZRX
1,1INCH Token,1inch,1INCH
2,1inch,1inch,1INCH
4,88mph V1,88mph,MPH
...,...,...,...
196,xSigma,xSigma,SIG
293,xToken,xToken,XTK
295,xvix,XVIX,XVIX
299,yAxis,yAxis,YAX


In [112]:
insert_debank_coin_cmc_symbol(df2)

['ZRX', '0x', '0x']
['1INCH', '1inch', '1inch']
['MPH', '88mph', '88mph V1']
['MPH', '88mph', '88mph V2']
['ADD', 'Add.xyz', 'ADD.XYZ']
['APY', 'APY.Finance', 'APY.Finance']
['AAVE', 'Aave', 'Aave V1']
['AAVE', 'Aave', 'Aave V2']
['ACS', 'ACryptoS', 'Acryptos']
['AST', 'AirSwap', 'Airswap']
['ALCX', 'Alchemix', 'Alchemix']
['ALPA', 'Alpaca City', 'Alpaca City']
['ALPACA', 'Alpaca Finance', 'Alpaca Finance']
['ALPHA', 'Alpha Homora', 'Alpha Homora']
['AMPL', 'Ampleforth', 'Ampleforth']
['ANY', 'Anyswap', 'AnySwap']
['ARMOR', 'Armor', 'Armor']
['AUTO', 'Auto', 'AutoFarm']
['BSWAP', 'BSCswap', 'BSCswap']
['BT', 'BT.Finance', 'BT.Finance']
['BADGER', 'Badger', 'Badger DAO']
['BAKE', 'BakeryToken', 'BakerySwap']
['BAND', 'Band Protocol', 'Band']
['BOND', 'BarnBridge', 'BarnBridge']
['BASE', 'Base Protocol', 'Base']
['BAC', 'Basis Cash', 'Basis Cash']
['BEL', 'Bella Protocol', 'Bella']
['BELT', 'Belt', 'Belt.fi']
['MARK', 'Benchmark', 'Benchmark(MARK)']
['BOND', 'Bonded Finance', 'Bonded']
[

['DDX', 'DerivaDAO', 'DerivaDAO']
['DEGO', 'Dego Finance', 'dego.finance']
['DEGOV', 'Degov', 'Degov']
['DEXTF', 'DEXTF Protocol', 'DEXTF Token']
['DF', 'dForce', 'dForce']
['DFD', 'DefiDollar DAO', 'DefiDollar DAO']
['DHT', 'dHedge DAO', 'dHedge DAO Token']
['DMG', 'DMM: Governance', 'DMM: Governance']
['DODO', 'DODO', 'DODO bird']
['DOKI', 'Doki Doki Finance', 'DokiDokiFinance']
['DOS', 'DEMOS', 'DOS Network Token']
['DOS', 'DOS Network', 'DOS Network Token']
['DOUGH', 'PieDAO DOUGH v2', 'PieDAO DOUGH v2']
['DRC', 'Digital Reserve Currency', 'Dracula Token']
['DRC', 'Dracula Token', 'Dracula Token']
['DRC', 'DRC mobility', 'Dracula Token']
['FARM', 'Harvest Finance', 'FARM Reward Token']
['FL', 'Freeliquid', 'FL']
['FOR', 'ForTube', 'The Force Token']
['FRM', 'Ferrum Network', 'Ferrum Network Token']
['FXS', 'Frax Share', 'Frax Share']
['GHST', 'Aavegotchi', 'Aavegotchi GHST Token']
['GNO', 'Gnosis', 'Gnosis Token']
['GOF', 'Golff', 'Golff.finance']
['GRO', 'Growth DeFi', 'Growth']
[

['SASHIMI', 'Sashimi', 'SashimiToken']
['SDT', 'Stake DAO', 'Stake DAO Token']
['SDT', 'Terra SDT', 'Stake DAO Token']
['SFI', 'saffron.finance', 'Spice']
['SIG', 'xSigma', 'xSigma']
['SNX', 'Synthetix', 'Synthetix Network Token']
['STBZ', 'Stabilize', 'Stabilize Token']
['SUSHI', 'SushiSwap', 'SushiToken']
['SWAP', 'TrustSwap', 'TrustSwap Token']
['SWISS', 'swiss.finance', 'Swiss Token']
['SWRV', 'Swerve', 'Swerve DAO Token']
['TORN', 'Tornado Cash', 'TornadoCash']
['TRU', 'TrueFi', 'TrustToken']
['UMA', 'UMA', 'UMA Voting Token v1']
['UNCX', 'UniCrypt', 'UniCrypt']
['UNN', 'UNION Protocol Governance Token', 'UNION Protocol Governance Token']
['VALUE', 'Value Liquidity', 'Value Liquidity']
['XVIX', 'XVIX', 'XVIX']
Traceback (most recent call last):
  File "<ipython-input-110-3675578a49a3>", line 8, in insert_debank_coin_cmc_symbol
  File "/Users/aarush/project/cripto_backtest/.venv/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  

In [113]:
df2[df2['symbol'] == 'ZRX']

,name_in_debank,name_in_cmc,symbol
0,0x,0x,ZRX
248,0x Protocol Token,0x,ZRX
